## Correos electronicos SPAM: Un enfoque con Procesamiento de Lenguaje Natural


### Uriel Antonio Alvarez Chavez 16111783

Los correos electronicos no deseados en su bandeja de entrada son molestos ya que perturban la rutina del usuario.Es por eso que las cuentas de correo electronico ya tiene un filtro de spam. Dadoo que es una de las aplicaciones PLN mas utilizadas vamos a ver como se derarrolla un filtro spam simple para correos electronicos.

In [1]:
from functools import reduce

import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import re

In [2]:
#Inserta los datos
full_corpus = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None, names =['label','msg_body'] )

#Separando los mensajes en 'ham'y 'spam'
ham_text=[]
spam_text=[]


### 1. Inspeccio y separacion de mensajes en las categorias "ham" y "spam"

Inicialmente el conjunto de datos debe inspeccionarce para ocuparlo y abordarlo para lograr la tarea. El formato de los datos ddos, la cantidad de datos proporcionados, la naturaleza de los datos se incluyen en esta inspeccion para identificar la mejor aproximacion posible para la tarea.

el corpus de menses dado ha marcado cada mensjae como ham o spam. Ademas hat 5568 mensajes en un Data Frame escrito en ingles que no son objetos nulos. Por lo tanto, el archivo  se puede leer usando DataFrame en python para clasificar esos mensajes de acuerdo con el indicador dado.

In [3]:
def separate_msgs():
    for index,column in full_corpus.iterrows():
        label=column[0]
        message_text=column[1]
        if label =='ham':
            ham_text.append(message_text)
        elif label == 'spam':
                spam_text.append(message_text)
                
separate_msgs()

### 2. Procesamiento de texto

El procesamiento es la tarea de realizar los pasos de preparacion den el corpus de texto sin formato para completar de manera eficiente una extraccion de texto o procesamiento de lenguaje natural o cualquier otra tarea que incluya texto sin formato. El procesamiento de texto consta de varios ppasos, aunque algunos de ellos pueden no aplicarce a una tarea en particular debido a la naturaleza del conjunto de datos disponible.

En esta tarea, el preprocesamiento de texto inlcuye los siguientes pasos de acuerdo con el conjunto de datos:

### Eliminacion de signos de puntuacion

In [4]:
# Eliminacion de los signos de puntuacion de los mensajes de correo electronico
def remove_msg_punctuations(email_msg):
    puntuation_remove_msg="".join([word for word in email_msg if word not in string.punctuation])
    return puntuation_removed_msg

### Convertir a minusculas:


Convertir a minusculas: La conversion de todos los caracteres del texto en un contexto comun, como los soportes en minusculas impide identificar dos palabaras de manera diferente donde una esta en minuscula y la otra no. Por ejemplo, "Primero" y "primero" deben identificarse como iguales, por lo tanto poner en minuscula todos los caracteres facilita la tarea.Ademas, las palabras de detencion tambien estan en minusculas, por lo que esto tambien haria posibles eliminar palabras de detencion mas adelante.

### Tokenizing

Tokenizing: La tokenizacion es la tarea de dividir el texto en partes significativas, es decir, tokens que incluyen oraciones y palabras. Un token se puede considerar como una instancia de una secuencia de caracteres en un texto particular que se agrupan para proporcionar una unidad semantica util para su posterior procesamiento. En esta tarea, de tokenizacion de palabras se raliza combinado espacios en blanco entre palabras como delimitador.Esto se logra en Python usando expresiones regulares para dividir una cadena en subcadenas con la funcion split(). que es un tokenizador basico.

In [5]:
#Convierte el texto en minusculas y tokenizing de palabras
def tokenize_into_words(text):
    tokens=re.split('\W+',text)
    return tokens

### Palabras


La derivacion es el proceso de eliminar afijos(sufijos, prefijos, infijos, circunfijos) de una palabra para obtener su raiz de palabra. Aunque la lematizacion esta relacionada con la derivacion, difiere ya que la lematizacion puede capturar formas canonicas basadas en el lema de una palabra. La lematizacion ocupa un vocabulario y un analisis morfologico de las palabras que lo hacen mas rapido y preciso que la derivacion. WordNetLemmatizer ha logrado la lematizacion en lenguaje Python.

In [6]:
#Lemmatizing
Word_lemmatizer = WordNetLemmatizer()
def lemmatization(tokenized_words):
    lemmatized_text = [word_lemmatizer.lemmatize(word)for word in tokenized_words]
    return ' '.join(lemmatized_text)

def preprocessing_msgs(corpus):
    categorized_text = pd.DataFrame(corpus)
    categorized_text['non_punc_message_body'] = categorized_text[0].apply(lambda msg: remove_msg_punctuations(msg))
    categorized_text['tokenized_msg_body'] = categorized_text['non_punc_message_body'].apply(lambda msg: tokenize_into_words(msg.lower()))
    categorized_text['lemmatized_msg_words'] = categorized_text['tokenized_msg_body'].apply(lambda word_list: lemmatization(word_list))
    return categorized_text['lemmatized_msg_words']

### 3. Extraccion de caracteristicas

Despues de la etapa de preprocesamiento, las caracteristicas deben extraerse del texto. Las caracteristicas son las unidades que admiten la tarea de clasificacion, y las bigrams son las carateristicas en esta tarea de clasificacion de mensajes. Los bigrams o las caracteristicas se extraen del texto preprocesamiento. inicialmenten, los unlgramasse adquieren , y luego esos unigramas se utilizan para obtener los unigramasen cada corpus ("ham" y "spam")

In [7]:
# Extraccion de caracteristicas. Ejemplo : n-grams
def feature_extraction(preprocessed_text):
    bigrams=[]
    unigrams_list=[]
    for msg in preprocessed_text:
        # agregando end of and start of al mensaje
        msg='<s>' +msg+'</s>'
        unigrams_list.append(msg.split())
    unigrams=[uni_list for sub_list in unigrams_list for uni_list in sub_list]
    bigrams.extend(nltk.bigrams(unigrams))
    return bigrams

### 4. Eliminacion de Stop Words

Hay ciertas palabras en un idioma (se utiliza ingles en la practica) que son necesarias para una oracion o una secuencia de palabras, aunque no contribuyena; significado de una frace considerada. La biblioteca de Kit de herramientas de llenguaje natural (NLTK) en Python proporciona palabras de detencion comunes para algunos idiomas.

En lugar de eliminar las palabras de detencion en el paso de preprocesamiento, se realiza despues de extraer las caracteristicas del corpus para evitar la ausencia de bigrams con palabras de una parada (('use', 'your'),('to','win')) ya que tienen un impacto en el resultado final de la aplicacion. Las palabras de detencion se pueden ignorar en esta Recuperacion de informacion orientada a palabras clave debido a su efecto en la precision de la recuperacion.

In [10]:
# Eliminando bigrams solo constop words
stopwords= nltk.corpus.stopwords.words('english')
def filter_stopwords_bigrams(bigram_list):
    filtered_bigrams=[]
    for bigram in bigram_list:
        if bigram[0] in stopwords and bigram[1] in stopwords:
            continue
        filtered_bigrams.append(bigram)
    return filtered_bigrams

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\uriel/nltk_data'
    - 'C:\\Users\\uriel\\anaconda3\\nltk_data'
    - 'C:\\Users\\uriel\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\uriel\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\uriel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


### 5. Obtener distribucion de frecuencia de caracteristicas

la distribucion de frecuencia se utiliza para obtener la frecuencia de aparicion de cada elemento de vocabulario en un texto determinado.

In [11]:
# Adquiriendo la frecuencia de caracteristica
def ham_bigram_feature_frequency():
    # Frecuencia de caracteristicas para mensajes ham
    ham_bigrams= feature_extraction(preprocessing_msgs(ham_text))
    ham_bigram_frequency = nltk.FreqDist(filter_stopwords_bigrams(ham_bigrams))
    return spam_bigram_frequency

### 6. Construyendo un modelo para la prediccion

el modelo para clasificar un mensaje dado como "ham" o "spam" se ha abordado calculado las probabilidades de bigram dentro de cada corpus. Inicialmente el mensaje dado debe procesarse previamente para avanzar con la clasificacion. Incluida la \eliminacion de signos de puntuacion el cambio de 

In [12]:
# Calculando probabilidades del bigram
def bigram_probability(message):
    probability_h = 1
    probability_s = 1
    # Preprocesando los mensaje de entrada
    punc_removed_message = "".join(word for word in message if word not in string.punctuation)
    punc_removed_message = '<s> ' +punc_removed_message +' </s>'
    tokenized_msg = re.split('\s+', punc_removed_message)
    lemmatized_msg = [word_lemmatizer.lemmatize(word)for word in tokenized_msg]
    # bigrams para el mensaje
    bigrams_for_msg = list(nltk.bigrams(lemmatized_msg))
    # Eliminamos stop words
    ham_unigrams = [word for word in feature_extraction(preprocessing_msgs(ham_text)) if word not in stopwords]
    spam_unigrams = [word for word in feature_extraction(preprocessing_msgs(spam_text)) if word not in stopwords]
    # Frecuencias de bigrams extraidas
    ham_frequency = ham_bigram_feature_frequency()
    spam_frequency  = spam_bigram_feature_frequency()
    print('========================== Calculando Probabilidades ==========================')
    print('----------- Frecuencias Ham ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        ham_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        ham_probability_of_bigram = ham_frequency[bigram] + 1
        print(bigram, ' ocurre ', ham_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_bigrams(ham_unigrams):
            ham_probability_denominator += 1
            if(first_unigram == bigram[0]):
                ham_probability_denominator += ham_frequency[first_unigram, second_unigram]
        probability = ham_probability_of_bigram / ham_probability_denominator
        probability_h *= probability
    print('\n')
    print('----------- Frecuencias Spam ------------')
    for bigram in bigrams_for_msg:
        # probabilidad de la primera palabra en bigram
        spam_probability_denominator = 0
        # probabilidad de bigram (suavizado) 
        spam_probability_of_bigram = spam_frequency[bigram] + 1
        print(bigram, ' ocurre ', spam_probability_of_bigram)
        for (first_unigram, second_unigram) in filter_stopwords_bigrams(spam_unigrams):
            spam_probability_denominator += 1
            if(first_unigram == bigram[0]):
                spam_probability_denominator += spam_frequency[first_unigram, second_unigram]
        probability = spam_probability_of_bigram / spam_probability_denominator
        probability_s *= probability
    print('\n')
    print('Probabilidad Ham: ' +str(probability_h))
    print('Probabildiad Spam: ' +str(probability_s))
    print('\n')
    if(probability_h >= probability_s):
        print('\"' +message +'\" es un mensaje Ham')
    else:
        print('\"' +message +'\" es un mensaje Spam')
    print('\n')
bigram_probability('Click here,  ..to win an iphone 11 pro max')
bigram_probability('Win a brand new car ')

NameError: name 'word_lemmatizer' is not defined